In [16]:
%env SPARK_VERSION=3.0.0 # TODO PUT YOUR VALUE

import sys
!{sys.executable} -m pip install pydeequ



env: SPARK_VERSION=3.0.0 # TODO PUT YOUR VALUE


In [17]:
import pydeequ

from pyspark.sql import SparkSession, Row
from pyspark import SparkConf, SparkContext, SQLContext


appName = "PySpark SQL Server Example - via JDBC"
master = "local[*]"
conf = SparkConf() \
.setAppName(appName) \
.setMaster(master) \
.set("spark.driver.extraClassPath","/home/jovyan/work/mssql-jdbc-8.4.1.jre11.jar") \
.set("spark.jars.packages",pydeequ.deequ_maven_coord) \
.set("spark.jars.excludes",pydeequ.f2j_maven_coord)
# mssql-jdbc-7.2.2.jre11.jar



sc = SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)



spark = sqlContext.sparkSession

print(sc.version)
# TODO create spark session with jdbc driver path

3.0.0


In [18]:
server_url = "host.docker.internal" # think what should be here
database_name = "AdventureWorks2012"
url = server_url + ";" + "databaseName=" + database_name + ";"



table = "Production.Product"
user = "sa" # your user creds here
password = "1ExoticFruit2/" # your user creds here



jdbcDF = spark.read.format("jdbc") \
.option("driver" , "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
.option("url", f"jdbc:sqlserver://host.docker.internal\\SQLEXPRESS;databaseName={database_name}") \
.option("dbtable", table) \
.option("user", user) \
.option("password", password) \
.load()



jdbcDF.show()
# TODO connect to DB with Spark using JDBC connection to read the data

+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-----------+----------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|MakeFlag|FinishedGoodsFlag| Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|      SellStartDate|SellEndDate|DiscontinuedDate|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-

In [19]:
### Data Analyzers section
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(spark) \
                 .onData(jdbcDF) \
                 .addAnalyzer(Size()) \
                 .addAnalyzer(Completeness("ProductNumber")) \    #to analyze how many null values in column
                 .addAnalyzer(CountDistinct("ProductNumber")) \   #to analyze number of disitnct values
                 .addAnalyzer(Mean("StandardCost")) \             #avarage cost of product
                 .addAnalyzer(Minimum("SafetyStockLevel")) \
                 .addAnalyzer(Maximum("SafetyStockLevel")) \
                 .run()

analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

# TODO analyze data here

SyntaxError: unexpected character after line continuation character (3818158348.py, line 7)

In [ ]:
### Data profiling section
from pydeequ.profiles import *

resultProfile = ColumnProfilerRunner(spark) \
                .onData(jdbcDF) \
                .run()

for col, profile in resultProfile.profiles.items():
    print(profile)
    
# TODO profile data here

In [ ]:
### Constraint Suggestions section
from pydeequ.suggestions import *

suggestionResult = ConstraintSuggestionRunner(spark) \
                   .onData(jdbcDF) \
                   .addConstraintRule(DEFAULT()) \
                   .run()

print(json.dumps(suggestionResult, indent = 2))

# TODO find meaninful constraints here

In [ ]:
### Constraint Verification section
import pandas as pd
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Warning, "Product Review Check")

checkResult = VerificationSuite(spark) \
              .onData(jdbcDF) \
              .addCheck(
                check.hasSize(lambda x: x >= 500) \
                .isUnique("ProductNumber") \
                .isNonNegative("StandardCost") \
                .isContainedIn("Class", ["L", "H", "M"])) \
              .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()

# TODO check selected constraints here and make beautify the report

In [ ]:
# write Dataframe with results into html file and copy it to local machine

import os
import shutil

report = checkResult_df.toPandas().to_html()
print(report)

file = open("/home/jovyan/work/index.html", 'w')
file.write(report)
file.close()

source = '/home/jovyan/work/index.html'
target = 'C:\\Users\\Anastasiya_Silivonch'

try:
    shutil.copy(source, target)
except IOError as e:
    print("Unable to copy file. %s" % e)
except:
    print("Unexpected error:", sys.exc_info())